# Data manipulation file after the first scraping step to expand the technical information column

## 1. Library and data import

In [1]:
import pandas as pd
import ast
from datetime import datetime
import re

In [2]:
# Loading the scraped data
df = pd.read_csv('final_scrape_2025.csv')
pd.set_option('display.max_columns', None)

In [3]:
# Displaying the data
df

,url,reference,title,subjects,key_players,key_events,technical_info,documentation_gateway,transparency,final_act
0,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0255(COD),Justice programme 2028-2034,7.40.02 Judicial cooperation in civil and comm...,{'European Parliament': [{'Joint committee res...,"{'03/09/2025': [{'Date': '03/09/2025', 'Event'...","{'Procedure reference': '2025/0255(COD)', 'Pro...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN
1,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0405(COD),Placing on the market of genetically modified ...,"3.10.09.06 Agro-genetics, GMOs, 4.60.02 Consum...",{'European Parliament': [{'Joint committee res...,"{'16/12/2025': [{'Date': '16/12/2025', 'Event'...","{'Procedure reference': '2025/0405(COD)', 'Pro...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN
2,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0540(COD),"Union support for asylum, migration and integr...",7.10 Free movement and integration of third-co...,{'European Parliament': [{'Committee responsib...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","{'Procedure reference': '2025/0540(COD)', 'Pro...","[{'Institution': 'European Commission', 'Docum...",{'Other Members': [{'Name': [{'text': 'ASENS L...,NaN
3,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0550(COD),'AgoraEU' programme 2028–2034,"1.20 Citizen's rights, 4.10.03 Child protectio...",{'European Parliament': [{'Joint committee res...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","{'Procedure reference': '2025/0550(COD)', 'Pro...","[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN
4,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0227(COD),Global Europe,"6.20 Common commercial policy in general, 8.70...",{'European Parliament': [{'Joint committee res...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","{'Procedure reference': '2025/0227(COD)', 'Pro...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
114,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0232(COD),Protection of workers from the risks related t...,"4.15.15 Health and safety at work, occupationa...",{'European Parliament': [{'Committee responsib...,"{'18/07/2025': [{'Date': '18/07/2025', 'Event'...","{'Procedure reference': '2025/0232(COD)', 'Pro...","[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN
115,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0074(COD),Extension of the timeframe for the establishme...,"7.30.30 Action to combat crime, 7.40.04 Judici...",{'European Parliament': [{'Committee responsib...,"{'02/04/2025': [{'Date': '02/04/2025', 'Event'...","{'Procedure reference': '2025/0074(COD)', 'Pro...","[{'Institution': 'European Commission', 'Docum...",NaN,{'pdfFinal act': '/oeil/en/procedure-file/pdf?...
116,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0056(COD),Common rules for imports: suspension of certai...,"6.20.02 Export/import control, trade defence, ...",{'European Parliament': [{'Committee responsib...,"{'07/03/2025': [{'Date': '07/03/2025', 'Event'...","{'Procedure reference': '2025/0056(COD)', 'Pro...","[{'Institution': 'European Commission', 'Docum...",NaN,{'pdfFinal act': '/oeil/en/procedure-file/pdf?...
117,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0404(COD),Simplifying and reducing the burden of the rul...,3.30.06 Information and communication technolo...,{'European Parliament': [{'Committee responsib...,"{'16/12/2025': [{'Date': '16/12/2025', 'Event'...","{'Procedure reference': '2025/0404(COD)', 'Pro...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN


## 2. Various manipulations and verifications

In [4]:
def safe_eval(val):
    """Convert string to dict if possible, else return empty dict."""
    if val == 'N/A':
        return {}
    try:
        return ast.literal_eval(val)
    except Exception:
        return {}

In [ ]:
def is_legislative_proposal(docs):
    if isinstance(docs, str):
        try:
            # Converts string in a list of dictionnaries
            docs = ast.literal_eval(docs)
        except Exception:
            return False

    if isinstance(docs, list) and len(docs) > 0:
        first_doc = docs[0]
        if isinstance(first_doc, dict) and 'Document type' in first_doc:
            return first_doc['Document type'] == 'Legislative proposal'
    return False

df['is_legislative_proposal'] = df['documentation_gateway'].apply(is_legislative_proposal)

In [ ]:
# Convert the technical information columns into a dict
df['technical_info'] = df['technical_info'].apply(safe_eval)

# Expand the technical info into separate columns for better accessibility and readability of the variables:
technical_df = pd.json_normalize(df['technical_info'])

# Combine with the original df
df = pd.concat([df.drop(columns=['technical_info']), technical_df], axis=1)

In [8]:
df

,url,reference,title,subjects,key_players,key_events,documentation_gateway,transparency,final_act,is_legislative_proposal,Procedure reference,Procedure type,Procedure subtype,Legislative instrument,Amendments and repeals,Legal basis,Stage reached in procedure,Committee dossier,Mandatory consultation of other institutions,Other legal basis
0,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0255(COD),Justice programme 2028-2034,7.40.02 Judicial cooperation in civil and comm...,{'European Parliament': [{'Joint committee res...,"{'03/09/2025': [{'Date': '03/09/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,False,2025/0255(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,"[{'text': 'Repealing Regulation 2021/693'}, {'...",Rules of Procedure EP 58 Treaty on the Functio...,Awaiting committee decision,CJ03/10/03804,NaN,NaN
1,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0405(COD),Placing on the market of genetically modified ...,"3.10.09.06 Agro-genetics, GMOs, 4.60.02 Consum...",{'European Parliament': [{'Joint committee res...,"{'16/12/2025': [{'Date': '16/12/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,2025/0405(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Directive,"[{'text': 'Amending Directive 2001/18/EC'}, {'...",Treaty on the Functioning of the EU TFEU 114 T...,Preparatory phase in Parliament,NaN,[{'text': 'European Economic and Social Commit...,NaN
2,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0540(COD),"Union support for asylum, migration and integr...",7.10 Free movement and integration of third-co...,{'European Parliament': [{'Committee responsib...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",{'Other Members': [{'Name': [{'text': 'ASENS L...,NaN,True,2025/0540(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,NaN,Rules of Procedure EP 58 Treaty on the Functio...,Awaiting committee decision,LIBE/10/03652,NaN,NaN
3,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0550(COD),'AgoraEU' programme 2028–2034,"1.20 Citizen's rights, 4.10.03 Child protectio...",{'European Parliament': [{'Joint committee res...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,True,2025/0550(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,"[{'text': 'Repealing Regulation 2021/818'}, {'...",Rules of Procedure EP 58 Rules of Procedure EP...,Awaiting committee decision,CJ65/10/03702,[{'text': 'European Economic and Social Commit...,NaN
4,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0227(COD),Global Europe,"6.20 Common commercial policy in general, 8.70...",{'European Parliament': [{'Joint committee res...,"{'16/07/2025': [{'Date': '16/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...",NaN,NaN,True,2025/0227(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Regulation,NaN,Rules of Procedure EP 58 Rules of Procedure EP...,Awaiting committee decision,CJ19/10/03697,"[{'text': 'European Court of Auditors', 'href'...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,https://oeil.europarl.europa.eu/oeil/en/proced...,2025/0232(COD),Protection of workers from the risks related t...,"4.15.15 Health and safety at work, occupationa...",{'European Parliament': [{'Committee responsib...,"{'18/07/2025': [{'Date': '18/07/2025', 'Event'...","[{'Institution': 'European Commission', 'Docum...","{'Rapporteurs, Shadow Rapporteurs and Committe...",NaN,True,2025/0232(COD),COD - Ordinary legislative procedure (ex-codec...,Legislation,Directive,"[{'text': 'Amending Directive 2004/37/EC'}, {'...",Treaty on the Functioning of the EU TFEU 153-p...,Awaiting committee decision,EMPL/10/03700,[{'text': 'European Econom

In [9]:
# Save the manipulated data
df.to_csv('final_sample_cod_manip_2025.csv')